In [40]:
import pickle
import pandas as pd
import os
import numpy as np
import torch

In [41]:
annotations_converter = pickle.load(open("/content/annotation_converter.pkl", "rb"))
annotations_csv_path = "/content/midlevel.chunks_90.csv"
df_annotations = pd.read_csv(annotations_csv_path, delimiter=',')

In [53]:
print(annotations_converter)

['closing_bottle', 'closing_door_inside', 'closing_door_outside', 'closing_laptop', 'drinking', 'eating', 'entering_car', 'exiting_car', 'fastening_seat_belt', 'fetching_an_object', 'interacting_with_phone', 'looking_or_moving_around (e.g. searching)', 'opening_backpack', 'opening_bottle', 'opening_door_inside', 'opening_door_outside', 'opening_laptop', 'placing_an_object', 'preparing_food', 'pressing_automation_button', 'putting_laptop_into_backpack', 'putting_on_jacket', 'putting_on_sunglasses', 'reading_magazine', 'reading_newspaper', 'sitting_still', 'taking_laptop_from_backpack', 'taking_off_jacket', 'taking_off_sunglasses', 'talking_on_phone', 'unfastening_seat_belt', 'using_multimedia_display', 'working_on_laptop', 'writing']


In [42]:
activity_index_map = {activity: index for index, activity in enumerate(annotations_converter)}
df_annotations['activity_index'] = df_annotations['activity'].map(activity_index_map)

In [62]:
for root, _, files in os.walk("/content/results"):
    for file in files:
        folder_name = os.path.basename(root)
        file_path = f"/content/results/{folder_name}/{file}"
        tensors = torch.load(file_path)
        filtered_annotations = df_annotations[
            df_annotations["file_id"]
            == f"{folder_name}/{os.path.splitext(file)[0]}"
        ]
        flattened_data = []
        max_frame = filtered_annotations["frame_end"].max()
        tensors_len = len(tensors)
        for _, row in filtered_annotations.iterrows():
            if pd.notna(row["activity_index"]):
                for frame in range(row["frame_start"], row["frame_end"] + 1):
                    if frame < tensors_len:
                        flattened_data.append(
                            {
                                "frame": frame,
                                "activity": row["activity"],
                                "activity_index": int(row["activity_index"]),
                                "predicted": annotations_converter[int(np.argmax(tensors[frame]))],
                                "predicted_index": np.argmax(tensors[frame]),
                            }
                        )
        flattened_df = pd.DataFrame(flattened_data)
        flattened_df.to_csv(f'/content/csv/{folder_name}-{os.path.splitext(file)[0]}.csv', index=False)
        matches = flattened_df['activity_index'] == flattened_df['predicted_index']
        accuracy = matches.sum() / len(flattened_df) * 100
        print(f"Accuracy for '{folder_name}/{os.path.splitext(file)[0]}' is {accuracy:.2f}%")

<ipython-input-62-3f4dae97fdb8>:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  tensors = torch.load(file_path)


Accuracy for 'vp12/run1_2018-05-24-15-44-28.kinect_color' is 84.98%
Accuracy for 'vp12/run2_2018-05-24-16-21-35.kinect_color' is 85.95%
Accuracy for 'vp15/run1_2018-05-30-13-05-35.kinect_color' is 75.14%
Accuracy for 'vp15/run2_2018-05-30-13-34-33.kinect_color' is 79.57%
Accuracy for 'vp14/run2_2018-05-30-10-42-33.kinect_color' is 71.79%
Accuracy for 'vp14/run1_2018-05-30-10-11-09.kinect_color' is 75.01%
Accuracy for 'vp13/run2_2018-05-30-11-34-54.kinect_color' is 74.25%
Accuracy for 'vp5/run1_2018-05-22-15-10-41.kinect_color' is 71.61%
Accuracy for 'vp5/run2b_2018-05-22-15-50-07.kinect_color' is 69.18%
Accuracy for 'vp8/run2_2018-05-23-15-30-27.kinect_color' is 87.13%
Accuracy for 'vp8/run1d_2018-05-23-14-54-38.kinect_color' is 89.70%
Accuracy for 'vp4/run2_2018-05-22-14-25-04.kinect_color' is 87.92%
Accuracy for 'vp4/run1_2018-05-22-13-28-51.kinect_color' is 88.54%
